# Session 10. Groupby Operations

### Grouped operations are a powerful way to aggregate, transform, and filter data. They rely on the mantra of “split–apply–combine:

### 1. Data is split into separate parts based on key(s).
### 2. A function is applied to each part of the data.
### 3. The results from each part are combined to create a new data set.
### This is a powerful concept because parts of your original data can be split up into independent parts to perform a calculation.  The split–apply–combine concept is also heavily used in “big data” systems that use distributed computing, with the data being split into independent parts and dispatched to a separate server where a function is applied, and the results then being combined together.




## This session will cover:

1. Groupby operations to aggregate, transform, and filter data
2. Built-in and custom user functions to perform groupby operations

In [1]:
import pandas as pd

In [27]:
import numpy as np

In [31]:
air_quality_no2 = pd.read_csv('https://raw.githubusercontent.com/thousandoaks/BEMM458/master/data/air_quality_no2_long.csv',parse_dates=True)

In [32]:
air_quality_pm25 = pd.read_csv('https://raw.githubusercontent.com/thousandoaks/BEMM458/master/data/air_quality_pm25_long.csv',parse_dates=True)

In [33]:
## the parameter axis=0 indicates concatenation by rows.
air_quality = pd.concat([air_quality_pm25, air_quality_no2], axis=0)

In [60]:
# let's remove location
air_quality.drop(['location'],axis=1,inplace=True)

In [93]:
# Let's cast date.utc as datetime type
air_quality['date.utc'] = pd.to_datetime(air_quality['date.utc'])

In [95]:
air_quality.set_index('date.utc',inplace=True)

In [96]:
air_quality.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3178 entries, 2019-06-18 06:00:00+00:00 to 2019-05-07 01:00:00+00:00
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   city       3178 non-null   object 
 1   country    3178 non-null   object 
 2   parameter  3178 non-null   object 
 3   value      3178 non-null   float64
 4   unit       3178 non-null   object 
dtypes: float64(1), object(4)
memory usage: 149.0+ KB


In [97]:
air_quality.head(10)

,city,country,parameter,value,unit
date.utc,,,,,
2019-06-18 06:00:00+00:00,Antwerpen,BE,pm25,18.0,µg/m³
2019-06-17 08:00:00+00:00,Antwerpen,BE,pm25,6.5,µg/m³
2019-06-17 07:00:00+00:00,Antwerpen,BE,pm25,18.5,µg/m³
2019-06-17 06:00:00+00:00,Antwerpen,BE,pm25,16.0,µg/m³
2019-06-17 05:00:00+00:00,Antwerpen,BE,pm25,7.5,µg/m³
2019-06-17 04:00:00+00:00,Antwerpen,BE,pm25,7.5,µg/m³
2019-06-17 03:00:00+00:00,Antwerpen,BE,pm25,7.0,µg/m³
2019-06-17 02:00:00+00:00,Antwerpen,BE,pm25,7.0,µg/m³
2019-06-17 01:00:00+00:00,Antwerpen,BE,pm25,8.0,µg/m³


In [98]:
air_quality.tail(10)

,city,country,parameter,value,unit
date.utc,,,,,
2019-05-07 11:00:00+00:00,London,GB,no2,21.0,µg/m³
2019-05-07 10:00:00+00:00,London,GB,no2,21.0,µg/m³
2019-05-07 09:00:00+00:00,London,GB,no2,28.0,µg/m³
2019-05-07 08:00:00+00:00,London,GB,no2,32.0,µg/m³
2019-05-07 07:00:00+00:00,London,GB,no2,32.0,µg/m³
2019-05-07 06:00:00+00:00,London,GB,no2,26.0,µg/m³
2019-05-07 04:00:00+00:00,London,GB,no2,16.0,µg/m³
2019-05-07 03:00:00+00:00,London,GB,no2,19.0,µg/m³
2019-05-07 02:00:00+00:00,London,GB,no2,19.0,µg/m³


# 1. Splitting a dataframe into groups

In [99]:
air_quality_grouped_by_parameter=air_quality.groupby('parameter')

In [100]:
air_quality_grouped_by_parameter.count()

,city,country,value,unit
parameter,,,,
no2,2068,2068,2068,2068
pm25,1110,1110,1110,1110


In [101]:
air_quality_grouped_by_city=air_quality.groupby('city')

In [102]:
air_quality_grouped_by_city.count()

,country,parameter,value,unit
city,,,,
Antwerpen,196,196,196,196
London,1978,1978,1978,1978
Paris,1004,1004,1004,1004


# 2. Applying basic functions to groups

### Once the GroupBy object has been created, several methods are available to perform a computation on the grouped data.

## 2.1. Basic functions applied to a single column

In [103]:
air_quality_grouped_by_city['value'].min()

city
Antwerpen    3.0
London       0.0
Paris        0.0
Name: value, dtype: float64

In [104]:
air_quality_grouped_by_city['value'].mean()

city
Antwerpen    23.576531
London       16.725480
Paris        27.740538
Name: value, dtype: float64

In [105]:
air_quality_grouped_by_city['value'].max()

city
Antwerpen    74.5
London       97.0
Paris        97.0
Name: value, dtype: float64

## 2.2. Multiple basic functions applied to a single column

In [106]:
air_quality_grouped_by_city['value'].agg([np.min, np.mean, np.max,np.std])

,amin,mean,amax,std
city,,,,
Antwerpen,3.0,23.576531,74.5,14.587113
London,0.0,16.725480,97.0,11.354230
Paris,0.0,27.740538,97.0,15.285746


In [109]:
air_quality_grouped_by_city['value'].resample('1D').mean()

city       date.utc                 
Antwerpen  2019-05-07 00:00:00+00:00    30.500000
           2019-05-08 00:00:00+00:00    18.000000
           2019-05-09 00:00:00+00:00    12.125000
           2019-05-10 00:00:00+00:00    11.750000
           2019-05-11 00:00:00+00:00    21.000000
                                          ...    
Paris      2019-06-17 00:00:00+00:00    34.395833
           2019-06-18 00:00:00+00:00    40.015385
           2019-06-19 00:00:00+00:00    26.950000
           2019-06-20 00:00:00+00:00    22.254545
           2019-06-21 00:00:00+00:00    20.000000
Name: value, Length: 135, dtype: float64

# 3. Applying generic functions to groups

# 3. Selecting a group

In [88]:
air_quality_grouped_by_city.get_group('Paris')

,city,country,date.utc,parameter,value,unit
0,Paris,FR,2019-06-21 00:00:00+00:00,no2,20.0,µg/m³
1,Paris,FR,2019-06-20 23:00:00+00:00,no2,21.8,µg/m³
2,Paris,FR,2019-06-20 22:00:00+00:00,no2,26.5,µg/m³
3,Paris,FR,2019-06-20 21:00:00+00:00,no2,24.9,µg/m³
4,Paris,FR,2019-06-20 20:00:00+00:00,no2,21.4,µg/m³
...,...,...,...,...,...,...
999,Paris,FR,2019-05-07 05:00:00+00:00,no2,72.4,µg/m³
1000,Paris,FR,2019-05-07 04:00:00+00:00,no2,61.9,µg/m³
1001,Paris,FR,2019-05-07 03:00:00+00:00,no2,50.4,µg/m³
1002,Paris,FR,2019-05-07 02:00:00+00:00,no2,27.7,µg/m³


In [72]:
air_quality_grouped_by_city.get_group('London')

,city,country,date.utc,parameter,value,unit
101,London,GB,2019-06-21 00:00:00+00:00,pm25,7.0,µg/m³
102,London,GB,2019-06-20 23:00:00+00:00,pm25,7.0,µg/m³
103,London,GB,2019-06-20 22:00:00+00:00,pm25,7.0,µg/m³
104,London,GB,2019-06-20 21:00:00+00:00,pm25,8.0,µg/m³
105,London,GB,2019-06-20 20:00:00+00:00,pm25,8.0,µg/m³
...,...,...,...,...,...,...
2063,London,GB,2019-05-07 06:00:00+00:00,no2,26.0,µg/m³
2064,London,GB,2019-05-07 04:00:00+00:00,no2,16.0,µg/m³
2065,London,GB,2019-05-07 03:00:00+00:00,no2,19.0,µg/m³
2066,London,GB,2019-05-07 02:00:00+00:00,no2,19.0,µg/m³


# 3. Iterate over groups

In [73]:
for name, group in air_quality_grouped_by_city:
   print('Hi there, I am in City:',name)
   print(group)
   print('\n')

Hi there, I am in City: Antwerpen
           city country                   date.utc parameter  value   unit
0     Antwerpen      BE  2019-06-18 06:00:00+00:00      pm25   18.0  µg/m³
1     Antwerpen      BE  2019-06-17 08:00:00+00:00      pm25    6.5  µg/m³
2     Antwerpen      BE  2019-06-17 07:00:00+00:00      pm25   18.5  µg/m³
3     Antwerpen      BE  2019-06-17 06:00:00+00:00      pm25   16.0  µg/m³
4     Antwerpen      BE  2019-06-17 05:00:00+00:00      pm25    7.5  µg/m³
...         ...     ...                        ...       ...    ...    ...
1094  Antwerpen      BE  2019-05-09 01:00:00+00:00       no2   20.0  µg/m³
1095  Antwerpen      BE  2019-05-08 02:00:00+00:00       no2   20.5  µg/m³
1096  Antwerpen      BE  2019-05-08 01:00:00+00:00       no2   23.0  µg/m³
1097  Antwerpen      BE  2019-05-07 02:00:00+00:00       no2   45.0  µg/m³
1098  Antwerpen      BE  2019-05-07 01:00:00+00:00       no2   50.5  µg/m³

[196 rows x 6 columns]


Hi there, I am in City: London
        c

# 4. Applying functions to groups

### Once the GroupBy object has been created, several methods are available to perform a computation on the grouped data.

In [13]:
groupedByContinent.size()

continent
Africa      624
Americas    300
Asia        396
Europe      360
Oceania      24
dtype: int64